In [47]:
%load_ext autoreload
%autoreload 2

import os, re, json, ast
from llm_api_calls import send_message_to_gemini_async, RateLimiter, send_message_open_router
from pprint import pprint
from algo import base_start_dialog_algorithm, base_continue_dialog_algorithm, base_end_dialog_algorithm, simulate_user_response, \
    student_card_template, send_message, \
    generate_readable_history_from_end, add_to_student_card, update_message_history_with_system_message, convert_gemini_history_to_open_router

rate_limiter = RateLimiter(13, 14)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# Start algo question answer scenario

In [48]:
# INIT DIALOG
usage_history = []
message_history = []

# Инициализация диалога, отправляем инструкцию и поулчаем ответ модели
student_card = student_card_template.copy()
prompted_message = base_start_dialog_algorithm.replace("$STUDENT_CARD", str(student_card))
message_history, usage_history = await send_message(prompted_message, message_history, usage_history, provider = 'router_sonnet')

{'input_tokens': 1427, 'output_tokens': 127, 'conversation_tokens': 1554}


USER:
 <SYSTEM_MESSAGE>Вы тактичный и вежливый ИИ-ассистент которого зовут Аика, вы помогаете ученику самостоятельно собрать алгоритм по распорядку дня для улучшения здоровья.
Вам дана json карточка ученика вместе с инструкцииями. Вам необходимо построить диалог таким образом, чтобы заполнить все поля в карточке по очереди задавая вопросы.
Вместо того, чтобы давать готовые решения, старайтесь использовать сократовский метод, задавая наводящие вопросы, чтобы помочь ученику самостоятельно создать свой собственный алгоритм дня.
Начните с первого вопроса про имя и возраст и далее следуйте по плану, задавая вопросы по одному.
Пиши только сообщения пользователю, как будто бы ты общаешься с ним через мессенджер. Будь вежливой, счастливой и остроумной. Не забывай уместно благодорить ученика за ответы.
Если все поля заполнены, то напишите кодовое слово FINISH и мы перейдем к следующему этапу.

Вот карточка ученика (зада

In [46]:
# TOODO если желчный удален то не спрашивать про желчегонную траву и желчегонную гимнастику


In [51]:
# Function to check if the dialog should terminate based on user query or system message.
def should_terminate(model_message):
    return 'terminate' in model_message.lower() or 'finish' in model_message.lower()

# Function to process and update dialog status based on the latest interaction.
async def process_dialog_interaction(user_query, student_card, dialog_count, message_history, usage_history):
    if user_query.strip() == '':
        user_query = "Расскажи шутку. А потом давай продолжим"
    
    prompted_message = user_query

    # Simulate sending and receiving message to/from a model
    message_history, usage_history = await send_message(prompted_message, message_history, usage_history, provider = 'router_sonnet')
    model_message = message_history[-1]['parts'][0]

    if (dialog_count % 4 == 0 and dialog_count > 0) or ('finish' in model_message.lower()):
        readable_history = await generate_readable_history_from_end(message_history, 5)
        sc_response = await add_to_student_card(readable_history=readable_history, 
                                                student_card=student_card, extraction_history=[], usage_history=[])
        student_card = sc_response['student_card']

    if should_terminate(model_message):
        return 'terminate', student_card

    return 'continue', student_card

# Main dialog loop
dialog_status = 'continue'
dialog_count = 0

try:
    while dialog_status != 'terminate':
        print('dialog_count:', dialog_count)

        # <Simulate user response>
        user_query = simulate_user_response.replace("$DOCTOR_QUESTION", message_history[-1]['parts'][0])
        response_user = await send_message_to_gemini_async(history = [{'role': 'user', 'parts': [user_query]}])
        user_query = response_user.get('text_response')
        # </Simulate user response>

        dialog_status, student_card = await process_dialog_interaction(user_query, student_card, dialog_count, message_history, usage_history)

        dialog_count += 1

except KeyboardInterrupt:
    print('KeyboardInterrupt')


dialog_count: 0
{'input_tokens': 1693, 'output_tokens': 130, 'conversation_tokens': 5022}


USER:
 Хорошо, Алиса. Какова ваша повседневная рутина?

\MODEL:
 Спасибо, Алиса, что поделились своим именем и возрастом. Отлично, теперь у меня есть эта базовая информация.

Следующий вопрос - какие цели по здоровью вы ставите перед собой на ближайшее время? Важно определить приоритеты, чтобы мы смогли вместе сформировать эффективный алгоритм для достижения этих целей.
dialog_count: 1
{'input_tokens': 1860, 'output_tokens': 170, 'conversation_tokens': 7052}


USER:
 Снизиться в весе, вести более активный образ жизни и улучшить свое общее самочувствие.

\MODEL:
 Благодарю вас, Алиса, за четкие и понятные цели по здоровью. Снижение веса, повышение активности и улучшение общего самочувствия - отличные приоритеты для работы над собой. 

Следующий вопрос - какие впечатления у вас остались от пройденного курса по здоровью? Это поможет мне лучше понять, какие аспекты курса были для вас наиболее полезн

In [73]:
student_card

{'description of card': {'value': 'карточка о здоровье, хранических заболеваниях, выбранном лечении'},
 'Имя и возраст': {'value': 'Алиса'},
 'Цели по здоровью': {'value': 'Снизиться в весе, вести более активный образ жизни и улучшить свое общее самочувствие'},
 'Промокод': {'value': '',
  'instruction': 'Спросите промокод, который он получил в девятом уроке. Он открывает доступ к формированию алгоритма. Без промокода доступ к алгоритму будет закрыт и Аика выйдет из этого сценария'},
 'Впечатления о курсе': {'value': 'Курс был очень полезным, особенно модули по диете и упражнениям. Информация была представлена в понятном виде, а практические советы можно легко внедрить в повседневную жизнь.'},
 'Артериальное давление': {'value': '120/80 мм рт. ст., нормотония'},
 'Тип сокращения желчного пузыря': {'value': 'нет камней в желчном пузыре'},
 'Трава желчегонная': {'value': 'тысячелистник'},
 'Желчегонная гимнастика': {'value': 'рекомендована'},
 'Уровень витамина Д в организме': {'value': 

In [41]:
message_history

[{'role': 'user',
  'content': "<SYSTEM_MESSAGE>Вы тактичный и вежливый ИИ-ассистент которого зовут Аика, вы помогаете ученику самостоятельно собрать алгоритм по распорядку дня для улучшения здоровья. \nВам по очереди дана json карточка ученика вместе с инструкцииями. Вам необходимо построить диалог таким образом, чтобы заполнить все обязательные поля в карточке. \nВместо того, чтобы давать готовые решения, старайтесь использовать сократовский метод, задавая наводящие вопросы, чтобы помочь ученику самостоятельно создать свой собственный алгоритм дня.\nНачните с первого вопроса про имя, дождись ответа пользователя, и далее следуйте по плану, задавая вопросы по одному, чтобы заполнить все незаполненные поля в карточке - спрашивай только 1 вопрос в одном сообщении. \nПиши только сообщения пользователю, как будто бы ты общаешься с ним через мессенджер. Будь вежливой, счастливой и остроумной. Не забывай уместно благодорить ученика за ответы.\nЕсли все поля заполнены, то напишите кодовое сло

# USAGES

In [71]:
# SONNET
import pandas as pd
us = pd.DataFrame(usage_history)
print("us['input_tokens'].sum()", us['input_tokens'].sum())
print("us['output_tokens'].sum()", us['output_tokens'].sum())
print("price", us['input_tokens'].sum()/1000 * 0.003 + us['output_tokens'].sum()/1000 * 0.015)
us

us['input_tokens'].sum() 47084
us['output_tokens'].sum() 2987
price 0.18605700000000003


,input_tokens,output_tokens,conversation_tokens
0,1427,127,1554
1,1561,84,3199
2,1693,130,5022
3,1860,170,7052
4,2068,97,9217
5,2238,241,11696
6,2522,202,14420
7,2780,200,17400
8,3007,212,20619
9,3237,233,24089


In [54]:
# GEMINI
us.to_csv('gemini_token_usage_16msgs.csv')
us

,input_tokens,output_tokens,conversation_tokens
0,979,47,1026
1,18,30,1074
2,4,51,1129
3,11,39,1179
4,14,87,1280
5,3,60,1343
6,5,86,1434
7,6,54,1494
8,4,141,1639
9,17,108,1764


{}